# 오늘의 목표

1. 표준국어대사전으로 만든 데이터프레임에서 명사 단어들만을 추출할 수 있다.
2. 한글로 표기된 단어가 주어졌을 때 각 음절의 유형을 추출할 수 있다.
    

## 예시

+ '왜냐하면' -> ('GV', 'CGV', 'CV', 'CGVC')

# 1. 데이터 살펴보기

## 1.1. 데이터프레임 가져오기

In [51]:
import pandas as pd

In [2]:
csvfilename = '../data/stdict_xml_20231105.csv'

In [3]:
df = pd.read_csv(csvfilename, index_col='target_code')
df

,word,word_unit,word_type,pos
target_code,,,,
1,가경-지,단어,한자어,명사
2,가계-하다01,단어,혼종어,동사
3,가계02,단어,한자어,명사
4,가계-되다,단어,혼종어,동사
5,가계-하다03,단어,혼종어,동사
...,...,...,...,...
534435,지체^장애,구,한자어,품사 없음
534436,신체^장애인,구,한자어,품사 없음
534437,지각06,단어,한자어,명사


## 1.2. 데이터 요약하기

In [4]:
# word_unit 칼럼의 값들을 세기
df['word_unit'].value_counts()

word_unit
단어     360629
구       62918
속담       7436
관용구      3887
Name: count, dtype: int64

In [5]:
# word_type 칼럼의 값들을 세기
df['word_type'].value_counts()

word_type
한자어    235445
혼종어     88548
고유어     75604
외래어     23950
Name: count, dtype: int64

In [6]:
df['pos'].value_counts()

pos
명사        268627
품사 없음      70562
동사         56238
형용사        12749
부사         11983
구          11323
어미           808
의존 명사        737
접사           533
감탄사          525
대명사          272
조사           174
수사           163
관형사          157
보조 형용사        11
보조 동사          8
Name: count, dtype: int64

## 1.3. 데이터의 범위를 선택하기

In [7]:
from collections import Counter
import re #정규표현식

In [8]:
# 데이터프레임의 word 칼럼에서 na 값들을 제외하고 str 자료형으로 설정하기
words = df['word'].dropna().astype('str') # EDIT THIS LINE

# 단어 목록에서 한글이 아닌 문자들이 출현한 횟수를 세기
nonhangul_cnt = Counter(re.findall(r'[^가-힣ㄱ-하-ㅣ]', ''.join(words))) # EDIT THIS LINE
print(sorted(nonhangul_cnt))

[' ', '(', ')', ',', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '[', ']', '^', '\ue13d', '\ue20c', '\uf537', '賈', '老', '綠', '復', '不', '良', '龍', '六', '利', '離', '狀']


위에서 살펴볼 수 있듯이 한글이 아닌 문자는 특수문자, 숫자, 한자가 대부분이지만, 여기 속하지 않는 수상한 문자도 있다.

In [9]:
spy = '\ue13d'
print(spy)




이 문자는 HWP 워드프로세서에서 함초롱 글꼴로 볼 수 있다.

# 2. 한글 음절 분석하기

## 2.1. 음절을 자모로 분해하기

In [10]:
from unicodedata import name, normalize

In [11]:
def decompose(str):
  # 문자열을 NFKD 방식으로 정규화하기

  return normalize('NFKD', str)

In [12]:
ng0 = 'ㅇ'
ng1 = decompose('응')
for ng in (ng0, ng1):
  for char in ng:
    print(char, name(char))

ㅇ HANGUL LETTER IEUNG
ᄋ HANGUL CHOSEONG IEUNG
ᅳ HANGUL JUNGSEONG EU
ᆼ HANGUL JONGSEONG IEUNG


In [13]:
def print_jamo_name(syl:str):
  print(syl, name(syl))
  for jamo in decompose(syl):
    print(jamo, name(jamo))

In [14]:
def romanize(hangul):
  # 한글 문자의 로마자 표기를 얻기
  hname = name(hangul)
  assert hname.startswith('HANGUL')
  return hname.split()[-1]

In [15]:
print(romanize('ㄱ')) #C

KIYEOK


In [16]:
print(romanize('ㅢ')) #GV (VG)

YI


In [17]:
print(romanize('ㅚ')) #GV

OE


In [18]:
print(romanize('ㅟ')) #GV

WI


## 2.2. 음절 유형 추출하기

In [19]:
import re

In [20]:
def is_letter(jamo):
    # 한글 문자가 LETTER 유형이면 True, 그렇지 않으면 False를 반환하기
    jname = name(jamo)
    return jname.startswith('HANGUL LETTER')

def is_choseong(jamo):
    # 한글 문자가 초성 유형이면 True, 그렇지 않으면 False를 반환하기'
    jname = name(jamo)
    return jname.startswith('HANGUL CHOSUNG')

def is_jungseong(jamo):
    # 한글 문자가 중성 유형이면 True, 그렇지 않으면 False를 반환하기
    jname = name(jamo)
    return jname.startswith('HANGUL JUNGSEONG')

def is_jongseong(jamo):
    # 한글 문자가 종성 유형이면 True, 그렇지 않으면 False를 반환하기
    jname = name(jamo)
    return jname.startswith('HANGUL JONGSUNG')

In [21]:
def get_onset_type(jamo):
  # LETTER 혹은 초성 유형의 문자의 유형을 반환하기
  # 자음 음가가 있으면 'C', 그렇지 않으면 ''를 반환하기
  return '' if romanize(jamo) == 'IEUNG' else 'C'

In [22]:
GLIDES = re.compile(r'^[WY]') # ^은 시작을 분명히 하기 위함

In [23]:
def get_nucleus_type(jamo):
  # LETTER 혹은 중성 유형의 문자의 유형을 반환하기
  # 단모음이면 'V', 활음이 있으면 'GV'를 반환하기, ㅚ(oe)는 아무도 단모음으로 발음하지 않으므로 이중모음으로 처리
  if GLIDES.search(romanize(jamo)) or romanize(jamo) == 'OE':
    return 'GV'
  else:
    return 'V'

In [24]:
def get_coda_type(jamo):
  # LETTER 혹은 종성 유형의 문자의 유형을 반환하기
  # 단자음이면 'C', 겹자음이면 'CC'를 반환하기
  return 'CC' if '-' in romanize(jamo) else 'C'


In [25]:
def is_composed_syllable(char):
  # 주어진 문자가 한글 1음절이면 True, 그렇지 않으면 False를 반환하는 함수
  try:
    return name(char).startswith('HANGUL SYLLABLE')
  except ValueError:
    return False

In [26]:
print(is_composed_syllable('앍'))
print(is_composed_syllable('ㅇ'))

True
False


In [27]:
print(is_composed_syllable(spy))

False


In [28]:
def is_syllable(strings):
  '''\
    >>> is_syllable('꺍')
    True
    >>> is is_syllable('ㄲㅑㄹ')
  '''
  # 주어진 문자열로 한글 1음절을 구성할 수 있으면 True, 그렇지 않으면 False를 반환하는 함수
  conjoined = normalize('NFKC', strings) #초성 중성 종성을 하나의 syllable로 만들기
  return is_composed_syllable(conjoined)


In [29]:
print(is_syllable('ㅇ'))
print(is_syllable('앍'))
print(is_syllable(decompose('앍')))

False
True
True


In [30]:
print(is_syllable(spy))

False


In [31]:
syl = '꺍'
dec = decompose(syl) # C/J/J 초성/중성/종성
onset, nucleus = (dec[0], dec[1]) # EDIT THIS LINE
coda = dec[2:]  # EDIT THIS LINE
print(onset, nucleus, coda)
print(get_onset_type(onset))

ᄁ ᅣ ᆰ
C


In [34]:
def get_syllable_types(string):
    # 문자열을 받아서 각 한글 문자들의 음절 유형으로 이루어진 튜플을 반환하는 함수
    return tuple(get_syllable_type(syl) for syl in string if is_syllable(syl))
    

In [52]:
def is_letter_syllable(strings):
    #모든 글자가 HANGUL LETTER인 경우
    output = ''
    try:   
        if all(is_letter(char) for char in strings):
            if len(strings) >= 2:
                onset, nucleus = strings[0:2]
            if len(strings) >= 3:
                coda = strings[2]
                onset = to_choseong(onset)
                nucleus = to_jungseong(nucleus)
                output += onset + nucleus
            if len(strings) >= 3:
                coda = strings[2]
                coda = to_jongseong(coda)
                output += coda
            return is_composed_syllable(normalize('NFKC', output))
    except:
        return False



In [35]:
get_syllable_types('얇은')

('CGVCC', 'CVC')

# 3. 표준국어대사전 표제어 단어들의 음절 유형 추출하기

In [36]:
df['syllable_types'] = None # EDIT THIS LINE
df

,word,word_unit,word_type,pos,syllable_types
target_code,,,,,
1,가경-지,단어,한자어,명사,None
2,가계-하다01,단어,혼종어,동사,None
3,가계02,단어,한자어,명사,None
4,가계-되다,단어,혼종어,동사,None
5,가계-하다03,단어,혼종어,동사,None
...,...,...,...,...,...
534435,지체^장애,구,한자어,품사 없음,None
534436,신체^장애인,구,한자어,품사 없음,None
534437,지각06,단어,한자어,명사,None


In [37]:
# 품사가 명사인 단어만 따로 저장하기
nouns_words = df[df['pos']] # EDIT THIS LINE

""


In [ ]:
nouns_syllables = nouns_words.explode('syllable_types')

In [38]:
# 음절 유형의 분포를 세어보기
nouns_syllables['syllable_types'].value_counts()

In [39]:
# 음절 유형의 분포를 단어 유형별로 각기 계산하기
table = pd.crosstab(nouns_syllables['word_type'], nouns_syllables['syllable_type'])
table

""


lamda <variable_name>:<return_value>=> 함수의 이름을 지정하지 않고 apply사용 가능

In [40]:
# 단어 유형별 음절 유형의 출현 빈도를 백분율로 표현하기
table.apply(lambda x: x/x.sum()*100, axis=1) # EDIT THIS LINEs

""
